## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page: [Medical Text Translation (EN-ES)](https://aws.amazon.com/marketplace/pp/prodview-lwpnc66c6bcug).
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Medical Text Translation (EN-ES)

- **Model**: `jsl-medical-translation-en-es`
- **Model Description**: Medical text translation between English (EN) and Spanish (ES). The model supports a maximum input length of 1024 tokens.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [4]:
model_name = "jsl-medical-translation-en-es"

real_time_inference_instance_type = "ml.g4dn.xlarge"
batch_transform_inference_instance_type = "ml.g4dn.xlarge"

## 2. create a deployable model from the model package.

In [5]:
# Define ModelPackage

model = ModelPackage(
    role=role,
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```


#### Important Parameters

- direction: str
   - Allowed values: ["en_to_es", "es_to_en"]
   - Default: "en_to_es"
You can specify these parameters in the input as follows:
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ],
    "direction": "en_to_es"
}
```

### 2.1 Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

#### A. Deploy the SageMaker model to an endpoint

In [6]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

----------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'


    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)
    
    input_data = json.dumps(data, indent=4, ensure_ascii=False) if content_type == 'application/json' else data


    if content_type == 'application/json':
        with open(input_file_name, "w", encoding='utf-8') as f:
            f.write(input_data)
    else:
        with open(input_file_name, 'w', encoding='utf-8') as file:
            for line in input_data.splitlines():
                json_object = json.loads(line)
                file.write(json.dumps(json_object, ensure_ascii=False) + '\n')

    with open(input_file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=data)

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4, ensure_ascii=False)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    with open(output_file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_output_dir}/{os.path.basename(output_file_name)}", Body=data)

    return response_data

#### Initial Setup

In [8]:
english_text = "Primary empty sella turcica associated with diabetes insipidus and campimetric defect"
english_records = [
    "The patient was diagnosed with acute myocardial infarction.",
    "Chronic obstructive pulmonary disease is characterized by long-term breathing problems.",
]


spanish_text = "Evaluación clínico-radiológica de la técnica de aloinjerto compactado y vástago cementado en cirugías de revisión de cadera."
spanish_records = [
    "El paciente fue sometido a una tomografía computarizada del abdomen.",
    "La hipertensión arterial es una condición crónica que puede causar daño a varios órganos."
]

#### JSON

##### Example 1

In [9]:
input_json_data = {"text": english_text, "direction": "en_to_es"}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json")

pd.DataFrame(data)

,predictions
0,Sella turca vacía primaria asociada a diabetes insípida y defecto campimétrico.


##### Example 2

In [10]:
input_json_data = {"text": spanish_text, "direction": "es_to_en"}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json")

pd.DataFrame(data)

,predictions
0,Clinico-radiological evaluation of the compact allo-graft and cemented stent technic in hip revision surgeries.


#### JSON Lines

In [11]:
import json

def create_jsonl(records, direction):
    if isinstance(records, str):
        records = [records]
    json_lines = '\n'.join(json.dumps({"text": text, "direction": direction}) for text in records)

    return json_lines


##### Example 1

In [12]:
input_jsonl_data = create_jsonl(english_text, direction= "en_to_es")

data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines")

print(data)

{"predictions": "Sella turca vacía primaria asociada a diabetes insípida y defecto campimétrico."}


##### Example 2

In [13]:
input_jsonl_data = create_jsonl(spanish_text, direction= "es_to_en")

data =  process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines")

print(data)

{"predictions": "Clinico-radiological evaluation of the compact allo-graft and cemented stent technic in hip revision surgeries."}


#### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [14]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

### 2.2 Batch inference

In [15]:
import json
import os



input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)



# JSON file names
validation_json_file_name1 = "input1.json"
validation_json_file_name2 = "input2.json"

# JSONL file names
validation_jsonl_file_name1 = "input1.jsonl"
validation_jsonl_file_name2 = "input2.jsonl"


# JSON paths
validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

# JSONL paths
validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"


def write_and_upload_to_s3(input_data, file_name):

    file_format = os.path.splitext(file_name)[1].lower()
    s3_key = f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}"

    if file_format == ".json":
        input_data_json = json.dumps(input_data, indent=4, ensure_ascii=False)

        with open(file_name, "w", encoding='utf-8') as f:
            f.write(input_data_json)
    elif file_format == ".jsonl":
        with open(file_name, 'w', encoding='utf-8') as file:
            for line in input_data.splitlines():
                json_object = json.loads(line)
                file.write(json.dumps(json_object, ensure_ascii=False) + '\n')

    with open(file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=s3_key, Body=data)

        
# Example data for JSONL files
input_jsonl_data1 = create_jsonl(english_records, direction= "en_to_es")
input_jsonl_data2 = create_jsonl(spanish_records, direction= "es_to_en")

# Example data for JSON files
input_json_data1 = {"text": english_records, "direction": "en_to_es"}
input_json_data2 = {"text": spanish_records, "direction": "es_to_en"}

# Write and upload JSONL data
write_and_upload_to_s3(input_jsonl_data1, f"{jsonl_input_dir}/{validation_jsonl_file_name1}")
write_and_upload_to_s3(input_jsonl_data2, f"{jsonl_input_dir}/{validation_jsonl_file_name2}")

# Write and upload JSON data
write_and_upload_to_s3(input_json_data1, f"{json_input_dir}/{validation_json_file_name1}")
write_and_upload_to_s3(input_json_data2,  f"{json_input_dir}/{validation_json_file_name2}")


#### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [17]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):
    
    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)

    with open(output_file_path, 'w', encoding='utf-8') as f_out:
        json.dump(data, f_out, indent=4, ensure_ascii=False)

In [18]:
# EN ==> ES
process_s3_json_output_and_save(validation_json_file_name1)

,predictions
0,El paciente fue diagnosticado de infarto agudo de miocardio.
1,La enfermedad pulmonar obstructiva crónica se caracteriza por problemas respiratorios a largo plazo.


In [19]:
# ES ==> EN
process_s3_json_output_and_save(validation_json_file_name2)

,predictions
0,The patient was subjected to computed tomography of the abdomen.
1,Arterial hypertension is a chronic condition that can cause damage to various organs.


#### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [22]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [23]:
# EN ==> ES
process_s3_jsonlines_output_and_save(validation_jsonl_file_name1)

{"predictions": "El paciente fue diagnosticado de infarto agudo de miocardio."}
{"predictions": "La enfermedad pulmonar obstructiva crónica se caracteriza por problemas respiratorios a largo plazo."}


In [24]:
# ES ==> EN
process_s3_jsonlines_output_and_save(validation_jsonl_file_name2)

{"predictions": "The patient was subjected to computed tomography of the abdomen."}
{"predictions": "Arterial hypertension is a chronic condition that can cause damage to various organs."}


In [25]:
model.delete_model()

INFO:sagemaker:Deleting model with name: jsl-medical-translation-en-es-1-2024-09-13-14-18-21-133


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

